In [1]:
import ipy_table
from collections import Counter
%run ../aux/postgres.py

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

import warnings
warnings.filterwarnings('ignore')

tknzr = TweetTokenizer()
#%run functions/preproc.py
#%run functions/classificacao.py

postgres functions loaded...


In [2]:
def tokensCleaner(txt):
    tokens = tknzr.tokenize(txt)
    stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
    clean_tokens = []
    for token in tokens:
        if token[:1] == '#':
            token = token[1:]
        if not token.isalpha():
            continue
        if token in stop:
            continue
        if token[:4] == 'http':
            continue
        if token[:1] == '@':
            continue
        if len(token) <= 3:
            continue

        clean_tokens.append(token)
    return clean_tokens

In [21]:
cmd = ("select cl.codclassificacao, cl.descricao, count(codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "where codresultadoanaliseprojeto = 19 "
       "GROUP BY 1 , 2"
      )
res = query(cmd)
suma = 0
for r in res:
    print(r)
    suma = suma + r[2]
print("\n",suma)

(227, 'personal', 34126)
(226, 'service', 151033)
(228, 'sale', 8613)
(229, 'clinical', 59598)
(223, 'information', 18103)
(224, 'product', 21833)

 293306


In [23]:
cmd = ("select "
       "cl.codclassificacao, "
       "cl.descricao,  to_char(datatweet, 'YYYY-MM') as data, "
       "count(r.codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "inner join tweet t ON r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 19 "
       "GROUP BY 1, 2, 3 "
       "ORDER BY 1, 2, 3"
      )
res = query(cmd)
res

[(223, 'information', '2016-08', 441),
 (223, 'information', '2016-09', 582),
 (223, 'information', '2016-10', 338),
 (223, 'information', '2016-11', 810),
 (223, 'information', '2016-12', 842),
 (223, 'information', '2017-01', 1035),
 (223, 'information', '2017-02', 438),
 (223, 'information', '2017-03', 636),
 (223, 'information', '2017-04', 458),
 (223, 'information', '2017-05', 400),
 (223, 'information', '2017-06', 644),
 (223, 'information', '2017-07', 453),
 (223, 'information', '2017-08', 403),
 (223, 'information', '2017-09', 1149),
 (223, 'information', '2017-10', 269),
 (223, 'information', '2017-11', 1382),
 (223, 'information', '2017-12', 279),
 (223, 'information', '2018-01', 510),
 (223, 'information', '2018-02', 261),
 (223, 'information', '2018-03', 857),
 (223, 'information', '2018-04', 728),
 (223, 'information', '2018-05', 195),
 (223, 'information', '2018-07', 137),
 (223, 'information', '2018-08', 83),
 (223, 'information', '2018-09', 332),
 (223, 'information', '

In [13]:
for r in res[:2]:
    print(r)
    print(r[1])
    print(r[2])
    # loading from database
    cmd = ("select t.text, cl.codclassificacao "
           "from resultadoanaliseprojetotweet r "
           "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
           "inner join tweet t on r.codtweet = t.codtweet "
           "where codresultadoanaliseprojeto = 19 and cl.codclassificacao = " + str(r[0]) + " "
           "AND to_char(datatweet, 'YYYY-MM') = '" + str(r[2]) + "'"
          )
    res01 = query(cmd)    
    print(cmd)
    print(len(res01))

(223, 'information', '2016-07', 175)
information
2016-07
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 12 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2016-07'
175
(223, 'information', '2016-08', 460)
information
2016-08
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 12 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2016-08'
460


In [24]:
for r in res:
    print(r)
    print(r[1])
    print(r[2])
    # loading from database
    cmd = ("select t.text, cl.codclassificacao "
           "from resultadoanaliseprojetotweet r "
           "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
           "inner join tweet t on r.codtweet = t.codtweet "
           "where codresultadoanaliseprojeto = 19 and cl.codclassificacao = " + str(r[0]))
    cmd = ("select t.text, cl.codclassificacao "
           "from resultadoanaliseprojetotweet r "
           "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
           "inner join tweet t on r.codtweet = t.codtweet "
           "where codresultadoanaliseprojeto = 19 and cl.codclassificacao = " + str(r[0]) + " "
           "AND to_char(datatweet, 'YYYY-MM') = '" + str(r[2]) + "'"
          )
    res01 = query(cmd)    
    print(cmd)
    print(len(res01))
    
    # concatenando texto
    txt = ''
    for row in res01:
        txt = txt + ' ' + row[0].lower()    
    
    # pré processamento
    clean_tokens = tokensCleaner(txt)
    
    # unigramas
    unigramas = ngrams(clean_tokens,1)
    unigramas_count = Counter(unigramas).most_common(10)
    arquivo = "data/unigramas_mes_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "a+") as f:
        for b in unigramas_count:
            f.write(r[2]+";"+b[0][0]+";"+str(b[1])+"\n")    
    
    # bigramas
    bigramas = ngrams(clean_tokens, 2)
    bigramas_count = Counter(bigramas).most_common(10)
    arquivo = "data/bigramas_mes_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "a+") as f:
        for b in bigramas_count:
            f.write(r[2]+";"+b[0][0]+";"+b[0][1]+";"+str(b[1])+"\n")    

    # trigramas
    trigramas = ngrams(clean_tokens, 3)
    trigramas_count = Counter(trigramas).most_common(10)
    arquivo = "data/trigramas_mes_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "a+") as f:
        for b in trigramas_count:
            f.write(r[2]+";"+b[0][0]+";"+b[0][1]+";"+b[0][2]+";"+str(b[1])+"\n")    
        
    print()
    

(223, 'information', '2016-08', 441)
information
2016-08
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2016-08'
441
data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2016-09', 582)
information
2016-09
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2016-09'
582
data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2016-10', 338)
information
2016-10
selec

data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2018-02', 261)
information
2018-02
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2018-02'
261
data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2018-03', 857)
information
2018-03
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2018-03'
857
data/unigramas_mes_information.csv
data/bigramas_mes_informati

select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2019-09'
168
data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2019-10', 172)
information
2019-10
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 223 AND to_char(datatweet, 'YYYY-MM') = '2019-10'
172
data/unigramas_mes_information.csv
data/bigramas_mes_information.csv
data/trigramas_mes_information.csv

(223, 'information', '2019-11', 141)
information
2019-11
select t.text, cl.codclassificacao from resultadoanaliseprojet

data/unigramas_mes_product.csv
data/bigramas_mes_product.csv
data/trigramas_mes_product.csv

(224, 'product', '2017-12', 865)
product
2017-12
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 224 AND to_char(datatweet, 'YYYY-MM') = '2017-12'
865
data/unigramas_mes_product.csv
data/bigramas_mes_product.csv
data/trigramas_mes_product.csv

(224, 'product', '2018-01', 911)
product
2018-01
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 224 AND to_char(datatweet, 'YYYY-MM') = '2018-01'
911
data/unigramas_mes_product.csv
data/bigramas_mes_product.csv
data/trigramas_mes_product.csv

(224, 'pr

select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 224 AND to_char(datatweet, 'YYYY-MM') = '2019-07'
167
data/unigramas_mes_product.csv
data/bigramas_mes_product.csv
data/trigramas_mes_product.csv

(224, 'product', '2019-08', 288)
product
2019-08
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 224 AND to_char(datatweet, 'YYYY-MM') = '2019-08'
288
data/unigramas_mes_product.csv
data/bigramas_mes_product.csv
data/trigramas_mes_product.csv

(224, 'product', '2019-09', 207)
product
2019-09
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on 

...

select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 227 AND to_char(datatweet, 'YYYY-MM') = '2019-03'
1481
data/unigramas_mes_personal.csv
data/bigramas_mes_personal.csv
data/trigramas_mes_personal.csv

(227, 'personal', '2019-04', 923)
personal
2019-04
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 227 AND to_char(datatweet, 'YYYY-MM') = '2019-04'
923
data/unigramas_mes_personal.csv
data/bigramas_mes_personal.csv
data/trigramas_mes_personal.csv

(227, 'personal', '2019-05', 914)
personal
2019-05
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classific

select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 228 AND to_char(datatweet, 'YYYY-MM') = '2017-06'
265
data/unigramas_mes_sale.csv
data/bigramas_mes_sale.csv
data/trigramas_mes_sale.csv

(228, 'sale', '2017-07', 224)
sale
2017-07
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 228 AND to_char(datatweet, 'YYYY-MM') = '2017-07'
224
data/unigramas_mes_sale.csv
data/bigramas_mes_sale.csv
data/trigramas_mes_sale.csv

(228, 'sale', '2017-08', 178)
sale
2017-08
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codcla

select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 228 AND to_char(datatweet, 'YYYY-MM') = '2019-02'
114
data/unigramas_mes_sale.csv
data/bigramas_mes_sale.csv
data/trigramas_mes_sale.csv

(228, 'sale', '2019-03', 55)
sale
2019-03
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 228 AND to_char(datatweet, 'YYYY-MM') = '2019-03'
55
data/unigramas_mes_sale.csv
data/bigramas_mes_sale.csv
data/trigramas_mes_sale.csv

(228, 'sale', '2019-04', 136)
sale
2019-04
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclass

data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinical.csv

(229, 'clinical', '2017-05', 1543)
clinical
2017-05
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 229 AND to_char(datatweet, 'YYYY-MM') = '2017-05'
1543
data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinical.csv

(229, 'clinical', '2017-06', 1816)
clinical
2017-06
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 229 AND to_char(datatweet, 'YYYY-MM') = '2017-06'
1816
data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinic

data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinical.csv

(229, 'clinical', '2018-12', 1241)
clinical
2018-12
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 229 AND to_char(datatweet, 'YYYY-MM') = '2018-12'
1241
data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinical.csv

(229, 'clinical', '2019-01', 1964)
clinical
2019-01
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 19 and cl.codclassificacao = 229 AND to_char(datatweet, 'YYYY-MM') = '2019-01'
1964
data/unigramas_mes_clinical.csv
data/bigramas_mes_clinical.csv
data/trigramas_mes_clinic

In [25]:
clean_tokens = tokensCleaner(txt)
print(clean_tokens[:10])

['best', 'option', 'beautiful', 'smile', 'great', 'option', 'replace', 'missing', 'teeth', 'dentist']


In [ ]:
clean_tokens = tokensCleaner(txt)

In [77]:
# unigramas
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
with open("data/unigramas.csv", "w") as f:
    for b in aaa:
        print("%s;%s" % (b[0][0], b[1] ))
        f.write(b[0][0]+";"+str(b[1])+"\n")

insurance;1952
handpiece;1435
abutment;973
angle;946
contra;910
surgical;835
cost;810
save;776
motor;743
reduction;720
check;647
speed;620
screw;566
surgery;495
bone;488
sales;481
system;475
money;455
straight;452
best;447
rica;432
costa;432
sponsored;422
idea;417
affordable;388
brushless;382
sale;371
getting;358
surprise;357
germany;356
analog;356
listings;356
taking;355
consider;341
importance;339
plunge;337
complete;301
tooth;293
free;286
internal;278
teeth;274
broken;268
cover;240
instruments;238
external;230
call;229
abutments;216
crown;212
model;210
drill;207
manager;202
sinus;196
online;196
lift;193
dentist;187
smile;186
nose;185
repair;176
remover;172
international;170
azdent;166
lanes;165
bowling;165
medical;163
removal;160
push;159
coxo;156
healing;154
expander;147
fractured;147
zimmer;140
machine;139
instrument;137
screen;136
rescue;134
health;129
endo;126
biomet;125
dublin;123
youtube;118
prices;117
learn;115
consultation;109
nobel;109
portland;104
demonstration;104
button;

ovarian;1
compromises;1
absolutely;1
dentalsurgeon;1
abaco;1
finest;1
marcos;1
lacrosse;1
utahn;1
dauphine;1
teenage;1
photographers;1
vanguard;1
forms;1
egypt;1
flexible;1
xmas;1
clock;1
corruption;1
alzheimer;1
nigga;1
molarcity;1
registration;1
launched;1
publish;1
nigeria;1
write;1
manesh;1
lakes;1
naviidentalcare;1
vitamins;1
cambridge;1
tips;1
mental;1
mirror;1
fresno;1
nikhil;1
esurance;1
clogged;1
amcoliandentalcare;1
morse;1
straightening;1
sore;1
expat;1
phones;1
dsnhealth;1
appeals;1
thrift;1
eastwood;1
blames;1
asian;1
beecher;1
hoyjte;1
beaute;1
canvas;1
duty;1
labels;1
awarded;1
uftf;1
hyundai;1
candle;1
difficulty;1
scrappy;1
samedayme;1
maxi;1
steelers;1
watering;1
anniversary;1
recovery;1
sugar;1
trouble;1
technique;1
superfan;1
especial;1
employment;1
asiana;1
campaign;1
cartridges;1
lense;1
inspection;1
detour;1
trees;1
environmental;1
dates;1
anatomy;1
citizen;1
dentallab;1
comprised;1
salesjobs;1
founded;1
neighbors;1
pantomography;1
cutter;1
error;1
crimson;1
bloo

In [71]:
# bigramas
ungs = ngrams(clean_tokens, 2)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s" % (b[0][0], b[0][1], b[1] ))

contra;angle;904
angle;handpiece;527
save;money;439
costa;rica;432
money;costa;415
rica;save;411
speed;contra;385
reduction;speed;379
idea;cost;358
sponsored;listings;356
cost;surprise;356
surprise;sponsored;356
importance;insurance;338
consider;taking;338
insurance;consider;338
taking;plunge;337
plunge;getting;337
getting;importance;317
abutment;analog;306
surgical;brushless;250
broken;screw;231
brushless;motor;218
listings;idea;205
sales;manager;200
sinus;lift;189
insurance;affordable;182
surgical;straight;173
straight;handpiece;172
handpiece;nose;171
check;bone;170
check;broken;166
insurance;bowling;165
straight;abutment;165
lanes;germany;165
bowling;lanes;165
international;sales;164
speed;handpiece;164
reduction;contra;157
germany;insurance;155
screw;remover;152
insurance;cover;152
bone;expander;143
handpiece;reduction;129
system;screen;123
affordable;insurance;117
screw;removal;117
angle;speed;105
motor;system;104
free;consultation;103
drill;motor;103
push;button;103
system;surgic

quality;manufacturers;2
slim;straight;2
patients;lauren;2
lift;scalers;2
enter;tech;2
repair;fractured;2
braces;debt;2
whatsapp;sale;2
screws;encourage;2
particularly;cringey;2
covered;abutment;2
check;soft;2
london;maidstone;2
bumper;bumper;2
buckinghamshire;sales;2
driver;traing;2
recall;clinician;2
handpiece;deals;2
drilling;screw;2
today;offer;2
instruments;implantation;2
speed;handpece;2
sale;bone;2
info;dentalclinic;2
surgery;benefit;2
recruiting;sales;2
check;sinus;2
parsippany;insurance;2
unique;part;2
four;ashes;2
button;aseptico;2
nozzle;reduce;2
replacement;free;2
best;care;2
finance;offer;2
analog;instruments;2
visit;injury;2
years;enough;2
midlands;helix;2
benefits;bone;2
nice;maryam;2
tooth;complete;2
abutment;interface;2
someone;needs;2
irrigation;oral;2
special;surgery;2
anatomic;abutments;2
something;life;2
handpiece;manufacturing;2
abutments;order;2
offering;surgical;2
kits;surgery;2
angle;slow;2
dentist;complete;2
interested;finance;2
better;still;2
attractive;succes

pricer;root;1
take;trip;1
spokane;downers;1
three;options;1
thread;coxo;1
abutments;idea;1
listings;paid;1
henderson;could;1
cover;ranking;1
disney;cruise;1
site;cherry;1
jersey;allendaledentalimplants;1
early;bird;1
sponsored;sign;1
create;prosthetics;1
instruments;world;1
office;today;1
dentalhealth;unique;1
pays;capitalism;1
partner;online;1
crown;karstadtquelle;1
free;thanked;1
intelligent;surgical;1
speed;profiles;1
call;care;1
yesterday;wife;1
part;screwing;1
abutment;ring;1
state;idea;1
systems;usually;1
bone;screws;1
manufacturers;astratech;1
starting;conditions;1
best;delay;1
jump;start;1
toronto;company;1
smile;idea;1
marylebone;details;1
mete;surgical;1
removal;system;1
grinding;teeth;1
basically;idea;1
coverage;need;1
knowledge;replace;1
lift;forget;1
headed;differently;1
abutment;hawaiifamilydental;1
dentist;already;1
sponsored;bone;1
swoon;take;1
covers;yabangbang;1
curios;locator;1
screen;insurance;1
thread;best;1
analog;scott;1
results;short;1
listed;includes;1
cement;r

pregnant;used;1
time;conscious;1
graduable;abutment;1
henry;ward;1
nederland;micromotor;1
sales;online;1
system;dump;1
alzheimer;disease;1
treatment;reach;1
charlotte;sales;1
savings;black;1
vitamin;crucial;1
pocket;probably;1
risks;muruga;1
special;fairfax;1
specialist;freiburg;1
sterile;check;1
smile;three;1
imagine;surgery;1
bone;preparation;1
sale;disease;1
debt;fast;1
diet;pill;1
done;youtube;1
offset;curved;1
porcelain;crowns;1
without;using;1
register;abutment;1
canal;sending;1
care;online;1
made;easy;1
upliftingnews;veteran;1
save;system;1
stores;online;1
lift;mark;1
abutments;short;1
system;instrument;1
idea;bone;1
save;third;1
removal;whether;1
crown;abutments;1
advantage;sale;1
used;cheap;1
optic;save;1
listings;post;1
shop;terpercaya;1
rehabilitation;style;1
sites;best;1
appointment;root;1
acrylic;bath;1
nozzle;robot;1
sameday;international;1
plans;helping;1
tooth;favorite;1
rica;july;1
tooth;machine;1
instrument;sterile;1
think;someone;1
coverage;attorney;1
nose;robot;1
ch

handpiece;lehigh;1
medwayjobs;titanium;1
biomet;certain;1
german;joel;1
site;sell;1
abortion;pricing;1
ankylos;root;1
biohorizons;machine;1
media;server;1
clinic;surgery;1
lose;special;1
screw;plastic;1
promoting;part;1
handpiece;sunday;1
filament;internet;1
site;congratulations;1
credit;covered;1
renovations;complete;1
child;custody;1
cover;robot;1
listings;wondering;1
demonstration;process;1
youtube;learn;1
quotes;best;1
viewing;boxes;1
free;makes;1
germany;million;1
results;successful;1
shop;demonstration;1
special;inclusive;1
sandent;broken;1
analog;site;1
sinha;offering;1
tonbridge;affordable;1
removal;analog;1
screw;part;1
single;affordable;1
sale;costa;1
presentations;coxo;1
people;time;1
vent;surgical;1
germany;metal;1
free;ball;1
options;root;1
prescott;abutment;1
dentures;fixed;1
races;malo;1
want;made;1
metal;free;1
listings;expensive;1
bike;insurance;1
chip;screw;1
tooth;yank;1
wiil;cost;1
covering;diego;1
jobsincrawley;crawleyjobs;1
loss;vending;1
thommen;importance;1
insu

sealed;coxo;1
instruments;tuesday;1
suck;magnet;1
luther;king;1
futurist;saeshin;1
coxo;surgical;1
system;single;1
makeover;interested;1
health;teethwhitening;1
insurance;lawyer;1
london;check;1
best;jersey;1
pick;covered;1
handpiece;bucks;1
soda;costing;1
bangkok;price;1
adblock;chrome;1
manufacturing;high;1
osseointegration;sales;1
country;usps;1
provides;patients;1
youtube;bali;1
balls;logo;1
aftr;took;1
medical;vision;1
ventures;insurance;1
abutment;lazer;1
cardiff;check;1
part;time;1
repair;service;1
biocare;nobelactive;1
square;totalimplantcom;1
regularly;learn;1
prosthetic;spaces;1
perfectteeth;appo;1
sports;cars;1
call;system;1
manufacturers;international;1
cheap;super;1
going;something;1
therapy;versus;1
lord;know;1
consultation;system;1
lone;star;1
specialist;affordable;1
teeth;look;1
huge;coming;1
hcsm;hiring;1
sale;recurrent;1
consultation;contact;1
screw;thanks;1
dentalimplants;well;1
package;idea;1
bags;surgery;1
irrigation;bone;1
alpha;abutment;1
thousand;oaks;1
offering

treatment;insurance;1
boulder;youtube;1
pieces;average;1
build;sell;1
probe;measuring;1
offices;best;1
upper;west;1
instruments;internal;1
twice;part;1
suffolk;mini;1
savings;thailand;1
restorations;prosthetic;1
cosmeticdentistry;affordable;1
emails;insurance;1
searchcom;youtube;1
find;surgery;1
working;deals;1
four;start;1
easy;loan;1
tooth;losing;1
years;imagine;1
introduction;provide;1
save;ailing;1
sign;midland;1
specializes;completing;1
ingroup;range;1
functioning;tooth;1
improvement;charged;1
crown;importance;1
hair;straightening;1
consultation;milltown;1
digitalhealth;reduction;1
thrift;shops;1
best;clinic;1
biddingwar;bidnow;1
canal;implantsinvisible;1
price;find;1
online;cosmetics;1
results;impressive;1
center;save;1
pieces;titanium;1
among;abutment;1
prices;dentistry;1
ideas;transition;1
specialist;brisbane;1
americans;thousands;1
group;idea;1
screw;best;1
code;develop;1
need;either;1
pilot;portland;1
magnet;fascinating;1
marketingideas;captivating;1
insurance;instead;1
plans

tooth;number;1
dentistry;abutments;1
houston;learn;1
straumann;catalog;1
ebay;fixation;1
cover;fiat;1
cauvery;emporium;1
mind;jamming;1
everything;gain;1
teeth;reach;1
interest;savvy;1
working;sandent;1
pays;medical;1
using;state;1
voucher;biohorizons;1
totalimplantcom;sponsored;1
overview;prosthetics;1
approximate;cost;1
india;bike;1
insurance;biomet;1
external;biohorizons;1
shoes;greece;1
fredericksburg;insurance;1
comparability;root;1
scan;abutment;1
vegas;nevada;1
sponsored;looking;1
sale;years;1
pieces;without;1
cruise;line;1
real;teeth;1
cheap;replacements;1
online;point;1
oakridge;titanium;1
alphabio;importance;1
screwdriver;tweet;1
sterile;abutment;1
online;cheap;1
need;insurance;1
service;printing;1
january;sale;1
representative;redmond;1
bumper;warranty;1
february;book;1
cheap;dentist;1
leave;unsightly;1
scrappy;bottles;1
done;sales;1
longines;online;1
quoted;saving;1
listings;made;1
onto;permanent;1
singapore;cost;1
bank;account;1
teeth;teeth;1
disposable;surgery;1
diet;plan

In [72]:
# trigramas
ungs = ngrams(clean_tokens, 3)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s;%s" % (b[0][0], b[0][1], b[0][2], b[1]))

contra;angle;handpiece;513
money;costa;rica;415
save;money;costa;414
rica;save;money;411
costa;rica;save;411
speed;contra;angle;385
idea;cost;surprise;356
cost;surprise;sponsored;356
surprise;sponsored;listings;356
importance;insurance;consider;338
insurance;consider;taking;338
consider;taking;plunge;337
taking;plunge;getting;337
reduction;speed;contra;320
plunge;getting;importance;317
getting;importance;insurance;317
sponsored;listings;idea;205
listings;idea;cost;205
surgical;straight;handpiece;172
straight;handpiece;nose;171
bowling;lanes;germany;165
insurance;bowling;lanes;165
international;sales;manager;163
check;broken;screw;158
reduction;contra;angle;156
lanes;germany;insurance;154
germany;insurance;bowling;154
surgical;brushless;motor;144
handpiece;reduction;speed;109
contra;angle;speed;105
brushless;motor;azdent;90
angle;speed;handpiece;88
nose;cone;external;88
handpiece;nose;cone;87
cone;external;nozzle;87
driving;instructor;insurance;86
affordable;driving;instructor;86
straig

opportunity;performing;sales;2
platform;straight;abutment;2
tooth;irrigation;disposable;2
provides;complete;range;2
working;reduction;speed;2
technology;save;smile;2
important;replace;missing;2
drivers;abutment;free;2
improve;health;smile;2
prices;healthcare;medical;2
shoes;online;shop;2
zimmer;reduction;speed;2
point;needed;cost;2
tooth;complete;arch;2
zimmer;biomet;considering;2
sponsored;listings;four;2
multiple;call;office;2
insurance;insurance;companies;2
cover;insurance;coverage;2
azdent;reduction;speed;2
instruments;sterile;tools;2
ratchet;wrench;abutment;2
abutment;analog;learn;2
academyawardsnews;lehigh;valley;2
angled;angulated;titanium;2
traditional;diameter;toothpick;2
method;check;broken;2
sponsored;listings;spring;2
tooth;many;teeth;2
metal;parts;straight;2
clinical;study;cases;2
insurance;root;canal;2
expander;boneexpanderkit;boneexpanderkitdental;2
method;complete;tools;2
front;tooth;subsequently;2
manhattan;manhattan;jobs;2
syringe;injector;instruments;2
clinic;maryleb

makeover;enter;micromotor;1
virginia;surgery;abutment;1
wrong;insurance;houses;1
endo;motor;congratulations;1
abutments;familyfun;covered;1
save;group;trip;1
shipping;covered;insurance;1
screen;family;attorney;1
cover;cost;bridal;1
youtube;bone;expanders;1
model;sale;surgical;1
pros;cons;guitar;1
lift;results;insurance;1
sponsored;oakridge;coxo;1
sponsored;listings;post;1
benefits;offer;without;1
bridges;miss;special;1
scan;expensive;insurance;1
saved;restore;lost;1
group;trip;costa;1
done;thre;september;1
know;insurance;details;1
covered;insurance;lawyer;1
gift;ideas;massages;1
states;sponsored;spring;1
shop;demonstration;model;1
patient;presents;days;1
dentistry;croatia;abutment;1
crown;included;harder;1
roofing;insurance;coverage;1
saturday;quote;henry;1
conta;angle;medical;1
manufacturing;ring;cent;1
news;parkbenchnews;save;1
handpiece;push;micromotor;1
today;sponsored;locator;1
system;surgical;brushles;1
preserves;bone;reality;1
free;dubai;online;1
science;manufacturer;korea;1
lmw

clinic;abutment;analog;1
identifying;need;sinus;1
smile;better;saving;1
rarely;pays;colorado;1
take;much;learned;1
dealers;special;single;1
biting;efficiency;sales;1
instruments;pieces;average;1
post;dodge;quad;1
caning;bare;surgical;1
smile;makeover;mdpimetals;1
kavo;ebaydailybids;check;1
based;manufacturer;soluti;1
handpiece;tuesday;quote;1
problem;hours;gives;1
working;broken;screw;1
abutment;analog;lock;1
omaha;offer;results;1
meal;medicaid;cover;1
take;longest;heal;1
results;astra;tech;1
specialist;jobs;توظيف;1
required;sales;denta;1
waste;removal;oxlife;1
dentist;usps;system;1
losing;tooth;traumatizing;1
crown;free;consultaton;1
sterile;tools;downey;1
texarkana;gazette;jawboneup;1
lost;removed;find;1
three;miss;special;1
high;instead;titanium;1
bone;grafting;reality;1
kent;sales;representative;1
insurance;individuals;truck;1
insurance;system;brushless;1
nissan;hybrids;sale;1
ship;secret;titanium;1
plunge;getting;drivers;1
publication;interface;humans;1
curios;needing;surgery;1
an

estimated;cost;insurance;1
bottoms;call;cujo;1
needed;amazing;deal;1
towards;complete;package;1
rica;replacement;lost;1
listings;screw;redone;1
spaces;left;coffee;1
months;importan;biohorizons;1
complete;procedure;contra;1
info;reduction;speed;1
bone;ridge;split;1
costa;rica;swot;1
insurance;maryland;denver;1
sale;thailand;check;1
abutment;analog;lots;1
bone;scrapers;instruments;1
lift;brushless;motor;1
dentist;free;consultation;1
gotta;sept;cash;1
abutment;analog;brief;1
motor;speed;push;1
best;places;free;1
analog;speed;handpiece;1
instruments;required;surgery;1
next;spring;insurance;1
supplier;whatsapp;tech;1
listings;metal;standard;1
standard;shipping;check;1
plunge;getting;call;1
usps;micromotor;surgical;1
kanani;kanani;group;1
northyork;artwork;sale;1
instruments;cost;called;1
smile;system;motor;1
wells;toothache;sponsored;1
bone;preparation;instruments;1
handpiece;machine;drill;1
india;sales;portland;1
instruments;bone;scraper;1
read;latest;infographic;1
creative;insurance;aroun

...

wanita;murah;malaysia;1
xive;wednesday;quote;1
price;veteran;youtube;1
special;save;full;1
connections;dentalimplants;save;1
online;shopping;mall;1
worldwide;check;fractured;1
district;glenwoodsmiles;save;1
fellow;yoga;student;1
time;minimal;discomfort;1
tools;kits;drills;1
abutment;order;coxo;1
insurance;individuals;troy;1
made;carotid;artery;1
lost;removed;smile;1
nissan;versa;speed;1
adhered;appropriately;save;1
takes;meet;kerr;1
removal;reduction;speed;1
budget;jobs;wordpress;1
twice;kids;according;1
jobsincardiff;amazing;special;1
video;sponsored;salinas;1
motor;handpiece;motor;1
companies;cover;teen;1
contact;lakeview;surgery;1
looking;system;single;1
henderson;opportunity;international;1
insurance;covers;system;1
manufacturing;hiring;bone;1
cost;best;insurance;1
ship;fracture;check;1
handpiece;medical;handpiece;1
cost;mini;america;1
thread;check;bone;1
cars;surgery;motor;1
repair;shop;annandale;1
save;replace;international;1
portland;sleep;dentistry;1
angle;handpiece;fiber;1
ess

sale;good;afford;1
sponsored;listings;fact;1
probably;covered;insurance;1
speed;mbps;manufacturing;1
turns;meant;abutment;1
slow;check;criminal;1
jobs;best;toronto;1
prophy;brush;polish;1
imaging;center;tsmmag;1
check;initial;drill;1
story;hiring;international;1
placement;surgery;abutment;1
restore;smile;nights;1
expansion;phoenix;sales;1
restaurant;cleaning;much;1
painlessly;insurance;cover;1
ship;sale;check;1
tarnished;media;spending;1
manager;hamburg;slow;1
mike;mango;system;1
navigation;assisted;simultaneous;1
system;screen;reduction;1
call;information;check;1
contributions;visual;foxpro;1
cover;abortion;pricing;1
boles;shop;online;1
edison;sponsored;oakridge;1
removal;totalimplantcom;check;1
scotia;spiral;parts;1
guide;aesthetics;insurance;1
motor;azdent;surgical;1
instrumentation;maximize;suitable;1
endo;system;best;1
cost;tooth;covered;1
live;gonna;give;1
managers;base;pricey;1
something;better;bitches;1
stick;candle;medicaid;1
angle;handpiece;years;1
listings;bottom;shelf;1
sta

sponsored;sign;office;1
internal;control;questionnaire;1
abutment;types;speed;1
insurance;landscape;company;1
trilobe;replica;slim;1
lovejobs;good;decision;1
manufacturing;high;quality;1
going;upload;looks;1
supply;tools;abutments;1
make;practice;management;1
universal;screw;removal;1
manufactured;high;success;1
smile;reduction;speed;1
main;germany;silicone;1
biocare;osseoset;motor;1
large;refund;discover;1
specialist;bavaria;germany;1
find;interesting;advertisements;1
crown;karstadtquelle;insurance;1
connection;singaporebizupdate;sells;1
reduction;territory;managers;1
google;free;advertising;1
reduce;cost;amazed;1
coverage;gateway;cemetery;1
awareness;month;lizard;1
less;competitors;abutment;1
cuba;prices;insurance;1
dentist;start;tooth;1
insurance;state;farm;1
free;mini;couple;1
covering;diego;pearless;1
abutment;explained;sales;1
used;replace;root;1
surgery;offer;ends;1
reduction;speed;push;1
list;part;palm;1
handpiece;super;know;1
warranty;insurance;rental;1
columbus;ohio;miami;1
i

sponsored;listings;spending;1
complete;astra;tech;1
beginning;journey;ontheblog;1
three;options;available;1
tank;refill;insurance;1
cons;minimalist;clothing;1
analog;anthogyr;manufacturer;1
angle;reducation;guerrilla;1
flipkarts;online;shopping;1
various;sizes;reduction;1
fall;scam;save;1
second;abutments;presentations;1
curios;bone;screws;1
restoration;smiletherapywpg;artwork;1
abutment;online;shop;1
drill;surgical;surgery;1
sponsored;listings;early;1
vacation;idea;cost;1
concave;osteotomes;sinus;1
resolve;foreign;corruption;1
cover;team;online;1
cost;mexico;much;1
mayspecial;single;pair;1
maintenance;take;international;1
case;looking;international;1
cost;call;office;1
france;business;class;1
fever;vaccine;effectiveness;1
cover;cost;federal;1
deposit;crown;dapto;1
achieve;best;smile;1
price;target;francs;1
yesterday;morning;motor;1
impladent;xive;rescue;1
azdent;handpiece;abutment;1
kind;give;brilliant;1
site;bone;expander;1
youtube;life;mortgage;1
manesh;fractured;root;1
complete;cro

complete;cost;best;1
minimu;restored;lost;1
irrigation;surgical;abutment;1
true;west;newyork;1
quotes;check;fractured;1
stable;look;maui;1
smile;idea;cost;1
insurance;specialpromo;double;1
westermeier;grow;affordable;1
screw;story;hiring;1
lasts;saturday;quote;1
working;root;canal;1
search;insurance;surgery;1
specialist;وظائف;توظيف;1
nick;powell;difficulty;1
tooth;save;smile;1
presentation;medical;device;1
screwwwed;seriously;actually;1
complete;price;wanted;1
right;check;broken;1
manufacturer;bredent;purchased;1
birmingham;international;airport;1
premium;vegas;tech;1
managers;base;peelmutualinsurance;1
parkbenchnews;save;colleagues;1
complete;carry;case;1
remover;affordable;expensive;1
landlord;policy;quote;1
quote;thomas;edison;1
germany;angular;abutments;1
later;special;surgery;1
biomet;check;biomet;1
pretty;good;idea;1
tools;bone;expender;1
rescue;totalimplant;screw;1
abutments;harder;specializes;1
shop;dubai;afford;1
combine;apply;today;1
internal;mini;type;1
tech;osseospeed;abutm

In [2]:
cmd = ("select t.text, cl.codclassificacao "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "inner join tweet t on r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 12 and cl.codclassificacao = 228 ")
res01 = query(cmd)

In [5]:
print(len(res01))

10308


In [4]:
res01[:10]

[('Coxo Dental Implant Contra Angle Handpiece 20:1 Reduction CX235 C6-9 CE  New  https://t.co/SQJHEQNG6X https://t.co/lErDpI1hkT',
  228),
 ('Check out Dental Implant Screw Remover repair kit #Totalimplant https://t.co/5IEJBC0Umx via @eBay',
  228),
 ('Check out Dental Implant Screw Remover &amp; Repair Kit #Totalimplant https://t.co/uyq2fRXVej via @eBay',
  228),
 ('Check out Dental Implant Screw Remover broken screw kit #Totalimplant https://t.co/OBMoXwAiCD via @eBay',
  228),
 ('Check out Dental Implant thread repair kit #Totalimplant https://t.co/h0DdOEt4Ys via @eBay',
  228),
 ('Check out Dental Implant thread repair set #Totalimplant https://t.co/xDGqmaVant via @eBay',
  228),
 ('Check out Dental Implant thread repair surgical kit #Totalimplant https://t.co/sVulgEoOiD via @eBay',
  228),
 ('Case in point: Kid #4 needed two dental implants. All in cost varied from $7200 to $15000! I got four different ‘quotes’ #hcldr',
  228),
 ('RT @ShimCode: Case in point: Kid #4 needed two dent

In [20]:
txt = ''
for r in res01:
    txt = txt + ' ' + r[0].lower()

In [25]:
print(txt[:1000])

 coxo dental implant contra angle handpiece 20:1 reduction cx235 c6-9 ce  new  https://t.co/sqjheqng6x https://t.co/lerdpi1hkt check out dental implant screw remover repair kit #totalimplant https://t.co/5iejbc0umx via @ebay check out dental implant screw remover &amp; repair kit #totalimplant https://t.co/uyq2frxvej via @ebay check out dental implant screw remover broken screw kit #totalimplant https://t.co/obmoxwaicd via @ebay check out dental implant thread repair kit #totalimplant https://t.co/h0ddoet4ys via @ebay check out dental implant thread repair set #totalimplant https://t.co/xdgqmavant via @ebay check out dental implant thread repair surgical kit #totalimplant https://t.co/svulgeooid via @ebay case in point: kid #4 needed two dental implants. all in cost varied from $7200 to $15000! i got four different ‘quotes’ #hcldr rt @shimcode: case in point: kid #4 needed two dental implants. all in cost varied from $7200 to $15000! i got four different ‘quotes’ #hcl… check out dental

In [45]:
aaaa = 'uga'
print(aaaa[1:])

ga


In [52]:
tokens = tknzr.tokenize(txt)
print(tokens[:10])
stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
clean_tokens = []
for token in tokens:
    if token[:1] == '#':
        token = token[1:]
    if not token.isalpha():
        continue
    if token in stop:
        continue
    if token[:4] == 'http':
        continue
    if token[:1] == '@':
        continue
    if len(token) <= 3:
        continue
        
    clean_tokens.append(token)
print(clean_tokens[:10])

['coxo', 'dental', 'implant', 'contra', 'angle', 'handpiece', '20:1', 'reduction', 'cx235', 'c6']
['coxo', 'contra', 'angle', 'handpiece', 'reduction', 'check', 'screw', 'remover', 'repair', 'totalimplant']


In [60]:
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
for b in aaa:
    print(b)

(('insurance',), 1952)
(('handpiece',), 1435)
(('abutment',), 973)
(('angle',), 946)
(('contra',), 910)
(('surgical',), 835)
(('cost',), 810)
(('save',), 776)
(('motor',), 743)
(('reduction',), 720)
(('check',), 647)
(('speed',), 620)
(('screw',), 566)
(('surgery',), 495)
(('bone',), 488)
(('sales',), 481)
(('system',), 475)
(('money',), 455)
(('straight',), 452)
(('best',), 447)
(('rica',), 432)
(('costa',), 432)
(('sponsored',), 422)
(('idea',), 417)
(('affordable',), 388)
(('brushless',), 382)
(('sale',), 371)
(('getting',), 358)
(('surprise',), 357)
(('germany',), 356)
(('analog',), 356)
(('listings',), 356)
(('taking',), 355)
(('consider',), 341)
(('importance',), 339)
(('plunge',), 337)
(('complete',), 301)
(('tooth',), 293)
(('free',), 286)
(('internal',), 278)
(('teeth',), 274)
(('broken',), 268)
(('cover',), 240)
(('instruments',), 238)
(('external',), 230)
(('call',), 229)
(('abutments',), 216)
(('crown',), 212)
(('model',), 210)
(('drill',), 207)
(('manager',), 202)
(('sinus

(('undergo',), 1)
(('checks',), 1)
(('policies',), 1)
(('progressive',), 1)
(('keypad',), 1)
(('enables',), 1)
(('pacemakers',), 1)
(('teamadc',), 1)
(('globe',), 1)
(('bradstreet',), 1)
(('receveur',), 1)
(('subscribe',), 1)
(('zigarette',), 1)
(('garage',), 1)
(('preparation',), 1)
(('bird',), 1)
(('oracle',), 1)
(('hipaa',), 1)
(('answered',), 1)
(('breakingnews',), 1)
(('winchester',), 1)
(('sound',), 1)
(('pfchangswish',), 1)
(('classified',), 1)
(('sachin',), 1)
(('nanocrystalline',), 1)
(('meeting',), 1)
(('printer',), 1)
(('comment',), 1)
(('surfaces',), 1)
(('spanish',), 1)
(('teetharenotcosmetic',), 1)
(('sturdy',), 1)
(('clearwater',), 1)
(('naviidental',), 1)
(('billing',), 1)
(('orthodontist',), 1)
(('bestdentistforimplantsinkolkata',), 1)
(('atlas',), 1)
(('cabal',), 1)
(('sreeleathers',), 1)
(('bacteria',), 1)
(('jump',), 1)
(('quad',), 1)
(('breeds',), 1)
(('brace',), 1)
(('desks',), 1)
(('hutchinson',), 1)
(('compact',), 1)
(('kinda',), 1)
(('broadwayfamilydentalpc',),

In [56]:
bgs = ngrams(clean_tokens,2)
aaa = Counter(bgs).most_common()
for b in aaa:
    print(b)

(('contra', 'angle'), 904)
(('angle', 'handpiece'), 527)
(('save', 'money'), 439)
(('costa', 'rica'), 432)
(('money', 'costa'), 415)
(('rica', 'save'), 411)
(('speed', 'contra'), 385)
(('reduction', 'speed'), 379)
(('idea', 'cost'), 358)
(('sponsored', 'listings'), 356)
(('cost', 'surprise'), 356)
(('surprise', 'sponsored'), 356)
(('importance', 'insurance'), 338)
(('consider', 'taking'), 338)
(('insurance', 'consider'), 338)
(('taking', 'plunge'), 337)
(('plunge', 'getting'), 337)
(('getting', 'importance'), 317)
(('abutment', 'analog'), 306)
(('surgical', 'brushless'), 250)
(('broken', 'screw'), 231)
(('brushless', 'motor'), 218)
(('listings', 'idea'), 205)
(('sales', 'manager'), 200)
(('sinus', 'lift'), 189)
(('insurance', 'affordable'), 182)
(('surgical', 'straight'), 173)
(('straight', 'handpiece'), 172)
(('handpiece', 'nose'), 171)
(('check', 'bone'), 170)
(('check', 'broken'), 166)
(('insurance', 'bowling'), 165)
(('straight', 'abutment'), 165)
(('lanes', 'germany'), 165)
(('bow

(('dentalimplants', 'placidway'), 2)
(('shipping', 'straight'), 2)
(('peel', 'care'), 2)
(('smile', 'check'), 2)
(('sponsored', 'polident'), 2)
(('starve', 'save'), 2)
(('majority', 'patients'), 2)
(('insurance', 'finally'), 2)
(('truly', 'lifechanging'), 2)
(('natural', 'looking'), 2)
(('abutments', 'plastic'), 2)
(('india', 'exact'), 2)
(('check', 'affordable'), 2)
(('broken', 'abutment'), 2)
(('best', 'thing'), 2)
(('save', 'crown'), 2)
(('care', 'insurance'), 2)
(('motor', 'style'), 2)
(('alone', 'crown'), 2)
(('plymouth', 'helix'), 2)
(('fort', 'worth'), 2)
(('results', 'diseases'), 2)
(('recruitment', 'jobsinplymouth'), 2)
(('growing', 'quality'), 2)
(('handpiece', 'tooth'), 2)
(('instrument', 'includes'), 2)
(('abutments', 'correctly'), 2)
(('handpiece', 'type'), 2)
(('surgical', 'guides'), 2)
(('coupons', 'please'), 2)
(('cost', 'nike'), 2)
(('torrance', 'sales'), 2)
(('lasts', 'straight'), 2)
(('allergy', 'remove'), 2)
(('financial', 'district'), 2)
(('analog', 'machine'), 2)


(('coverage', 'luminous'), 1)
(('best', 'made'), 1)
(('coverage', 'making'), 1)
(('edition', 'houston'), 1)
(('months', 'straumann'), 1)
(('ship', 'speed'), 1)
(('placement', 'tool'), 1)
(('blames', 'losing'), 1)
(('travelling', 'sales'), 1)
(('idea', 'dentallab'), 1)
(('crown', 'motor'), 1)
(('intense', 'pheealzaimplant'), 1)
(('included', 'worth'), 1)
(('beck', 'verlag'), 1)
(('comfort', 'really'), 1)
(('direct', 'swish'), 1)
(('motorhandpiece', 'root'), 1)
(('site', 'four'), 1)
(('canada', 'tooth'), 1)
(('whether', 'looking'), 1)
(('prices', 'vary'), 1)
(('tool', 'standard'), 1)
(('brochure', 'pages'), 1)
(('daily', 'supplies'), 1)
(('maker', 'overweight'), 1)
(('canada', 'online'), 1)
(('bought', 'last'), 1)
(('analog', 'mini'), 1)
(('getting', 'back'), 1)
(('inspection', 'checklist'), 1)
(('review', 'motor'), 1)
(('shopping', 'costs'), 1)
(('manufactures', 'keyword'), 1)
(('answered', 'newly'), 1)
(('leonard', 'gordon'), 1)
(('dentist', 'irfan'), 1)
(('sniping', 'favorite'), 1)
((

(('electromagnetic', 'fields'), 1)
(('know', 'priced'), 1)
(('sponsored', 'scottsdale'), 1)
(('jinme', 'reduction'), 1)
(('sheth', 'completes'), 1)
(('crown', 'kensington'), 1)
(('legit', 'loans'), 1)
(('allonfourimplants', 'premium'), 1)
(('slovenia', 'turns'), 1)
(('india', 'save'), 1)
(('cost', 'robot'), 1)
(('martin', 'hotels'), 1)
(('azdent', 'waiting'), 1)
(('tooth', 'pasadena'), 1)
(('placed', 'health'), 1)
(('death', 'comment'), 1)
(('frustrating', 'insurance'), 1)
(('crimson', 'ventures'), 1)
(('whatts', 'manufactures'), 1)
(('sameday', 'sameday'), 1)
(('motor', 'market'), 1)
(('angle', 'broken'), 1)
(('pedals', 'insurance'), 1)
(('insurance', 'used'), 1)
(('lock', 'attachment'), 1)
(('book', 'abutment'), 1)
(('truck', 'transmission'), 1)
(('shoulder', 'regional'), 1)
(('fullmouthrehabilit', 'learn'), 1)
(('sprint', 'cheap'), 1)
(('azdent', 'mini'), 1)
(('smile', 'houston'), 1)
(('cover', 'metlife'), 1)
(('insurance', 'around'), 1)
(('zimmer', 'save'), 1)
(('analog', 'wants'),

(('flooring', 'wichita'), 1)
(('solution', 'trust'), 1)
(('fullmouthrehabilita', 'titanium'), 1)
(('curios', 'sale'), 1)
(('behind', 'trying'), 1)
(('past', 'months'), 1)
(('super', 'glad'), 1)
(('crown', 'screw'), 1)
(('month', 'book'), 1)
(('thommen', 'astra'), 1)
(('businesslisting', 'businessindia'), 1)
(('schools', 'offer'), 1)
(('paid', 'periodontist'), 1)
(('first', 'sure'), 1)
(('complete', 'bone'), 1)
(('abutment', 'phoenix'), 1)
(('denture', 'placed'), 1)
(('anatomical', 'screw'), 1)
(('machu', 'picchu'), 1)
(('straight', 'drills'), 1)
(('find', 'call'), 1)
(('tooth', 'traumatizing'), 1)
(('renewal', 'micromotor'), 1)
(('cover', 'hollow'), 1)
(('crowns', 'layered'), 1)
(('instruments', 'medical'), 1)
(('seasons', 'loans'), 1)
(('minute', 'budget'), 1)
(('azdent', 'machine'), 1)
(('area', 'start'), 1)
(('clinic', 'travel'), 1)
(('transmission', 'repair'), 1)
(('bifocal', 'distraction'), 1)
(('homemade', 'coca'), 1)
(('pharma', 'surgery'), 1)
(('free', 'consultaton'), 1)
(('hig

(('insurance', 'waste'), 1)
(('presentation', 'medical'), 1)
(('best', 'chesterfield'), 1)
(('manufacturers', 'savings'), 1)
(('machine', 'smart'), 1)
(('tools', 'worried'), 1)
(('lasts', 'surgic'), 1)
(('albany', 'artwork'), 1)
(('tool', 'screw'), 1)
(('back', 'best'), 1)
(('cover', 'linear'), 1)
(('insurance', 'braces'), 1)
(('insurance', 'welcome'), 1)
(('well', 'idea'), 1)
(('kaba', 'keypad'), 1)
(('june', 'astra'), 1)
(('system', 'trumptaxreturns'), 1)
(('endostealimplant', 'epostealimplant'), 1)
(('retail', 'addition'), 1)
(('already', 'paid'), 1)
(('contemporary', 'centre'), 1)
(('days', 'week'), 1)
(('miracas', 'online'), 1)
(('energy', 'good'), 1)
(('treatment', 'system'), 1)
(('biomet', 'kits'), 1)
(('free', 'hiossen'), 1)
(('coxo', 'paying'), 1)
(('adin', 'system'), 1)
(('aaid', 'dentalimplants'), 1)
(('using', 'dublin'), 1)
(('monoline', 'earthquake'), 1)
(('honest', 'world'), 1)
(('cost', 'boone'), 1)
(('employees', 'using'), 1)
(('expert', 'single'), 1)
(('five', 'offer')

(('check', 'abutment'), 1)
(('instruments', 'publication'), 1)
(('cicada', 'medical'), 1)
(('handpiece', 'convenient'), 1)
(('essential', 'save'), 1)
(('youtube', 'beaver'), 1)
(('canal', 'goes'), 1)
(('implantology', 'instruments'), 1)
(('surgery', 'placemeat'), 1)
(('parts', 'system'), 1)
(('contact', 'bravo'), 1)
(('boobs', 'insurance'), 1)
(('internal', 'originl'), 1)
(('term', 'care'), 1)
(('replaces', 'lost'), 1)
(('congratulations', 'ryan'), 1)
(('credit', 'online'), 1)
(('cover', 'washdcnews'), 1)
(('live', 'sports'), 1)
(('coming', 'invisalign'), 1)
(('sites', 'portland'), 1)
(('luckily', 'cover'), 1)
(('watches', 'online'), 1)
(('proceudres', 'experienced'), 1)
(('boxing', 'classes'), 1)
(('surgery', 'double'), 1)
(('losing', 'lost'), 1)
(('temperature', 'save'), 1)
(('bicon', 'trinia'), 1)
(('allege', 'installs'), 1)
(('range', 'place'), 1)
(('instruments', 'reduction'), 1)
(('blouses', 'online'), 1)
(('cost', 'approximately'), 1)
(('damn', 'glad'), 1)
(('dentist', 'drivers'

In [58]:
tgs = ngrams(clean_tokens,3)
aaa = Counter(tgs).most_common()
for b in aaa:
    print(b)

(('contra', 'angle', 'handpiece'), 513)
(('money', 'costa', 'rica'), 415)
(('save', 'money', 'costa'), 414)
(('rica', 'save', 'money'), 411)
(('costa', 'rica', 'save'), 411)
(('speed', 'contra', 'angle'), 385)
(('idea', 'cost', 'surprise'), 356)
(('cost', 'surprise', 'sponsored'), 356)
(('surprise', 'sponsored', 'listings'), 356)
(('importance', 'insurance', 'consider'), 338)
(('insurance', 'consider', 'taking'), 338)
(('consider', 'taking', 'plunge'), 337)
(('taking', 'plunge', 'getting'), 337)
(('reduction', 'speed', 'contra'), 320)
(('plunge', 'getting', 'importance'), 317)
(('getting', 'importance', 'insurance'), 317)
(('sponsored', 'listings', 'idea'), 205)
(('listings', 'idea', 'cost'), 205)
(('surgical', 'straight', 'handpiece'), 172)
(('straight', 'handpiece', 'nose'), 171)
(('bowling', 'lanes', 'germany'), 165)
(('insurance', 'bowling', 'lanes'), 165)
(('international', 'sales', 'manager'), 163)
(('check', 'broken', 'screw'), 158)
(('reduction', 'contra', 'angle'), 156)
(('lan

(('common', 'prosthetic', 'complication'), 2)
(('insurance', 'plans', 'insurance'), 2)
(('straight', 'abutment', 'internal'), 2)
(('zimmer', 'totalimplantcom', 'check'), 2)
(('abutment', 'improves', 'stability'), 2)
(('compression', 'check', 'bone'), 2)
(('free', 'shipping', 'artwork'), 2)
(('explaining', 'abutments', 'bridges'), 2)
(('whatsapp', 'bone', 'mill'), 2)
(('entertainment', 'washer', 'dryer'), 2)
(('complete', 'smile', 'portland'), 2)
(('screen', 'system', 'motor'), 2)
(('castable', 'abutment', 'trilobe'), 2)
(('check', 'bone', 'tack'), 2)
(('disease', 'tooth', 'fashion'), 2)
(('germany', 'smile', 'makeover'), 2)
(('mini', 'driver', 'self'), 2)
(('finance', 'offer', 'idea'), 2)
(('abutment', 'concave', 'healing'), 2)
(('find', 'abutment', 'analog'), 2)
(('temporary', 'straight', 'peek'), 2)
(('supplier', 'whatsapp', 'bone'), 2)
(('representative', 'alabama', 'florida'), 2)
(('save', 'wowcher', 'ceramic'), 2)
(('results', 'diseases', 'tricky'), 2)
(('surgical', 'tapered', 'sc

(('webinar', 'placing', 'innovative'), 1)
(('disability', 'determination', 'services'), 1)
(('mumbai', 'based', 'online'), 1)
(('internal', 'woodlands', 'sales'), 1)
(('cicada', 'interested', 'international'), 1)
(('store', 'identifying', 'need'), 1)
(('thailand', 'risks', 'thrift'), 1)
(('every', 'year', 'smart'), 1)
(('growth', 'dentistry', 'anti'), 1)
(('ideas', 'learn', 'affordable'), 1)
(('whatsapp', 'tech', 'speeds'), 1)
(('california', 'bone', 'expander'), 1)
(('call', 'affordable', 'affordable'), 1)
(('standard', 'reducation', 'contra'), 1)
(('fast', 'shipping', 'fractured'), 1)
(('assessment', 'prices', 'clinicalapprovals'), 1)
(('sale', 'motor', 'surgery'), 1)
(('upfront', 'insurance', 'forc'), 1)
(('behind', 'trying', 'avoid'), 1)
(('premium', 'greensboro', 'mike'), 1)
(('french', 'bulldog', 'dentistry'), 1)
(('dollars', 'offering', 'complete'), 1)
(('particularly', 'cringey', 'system'), 1)
(('complete', 'travelling', 'disposable'), 1)
(('hose', 'already', 'involved'), 1)
((

(('surgery', 'anterior', 'custom'), 1)
(('caps', 'megagenâ', 'abutments'), 1)
(('companies', 'idea', 'cost'), 1)
(('parkbenchnews', 'surgery', 'drill'), 1)
(('safe', 'auto', 'insurance'), 1)
(('drills', 'medical', 'devices'), 1)
(('analog', 'kits', 'sale'), 1)
(('sizes', 'surgical', 'system'), 1)
(('cost', 'insurance', 'apply'), 1)
(('shipping', 'sponsored', 'bone'), 1)
(('straumann', 'washington', 'patient'), 1)
(('modern', 'dentistry', 'learn'), 1)
(('unit', 'push', 'button'), 1)
(('free', 'insurance', 'cover'), 1)
(('free', 'ship', 'bone'), 1)
(('financing', 'options', 'motor'), 1)
(('handpiece', 'bags', 'surgery'), 1)
(('office', 'space', 'tampa'), 1)
(('analog', 'machine', 'surgical'), 1)
(('endo', 'motor', 'kits'), 1)
(('pull', 'giant', 'dustpan'), 1)
(('take', 'whole', 'family'), 1)
(('screws', 'totalimplantcom', 'check'), 1)
(('insurance', 'individuals', 'porsche'), 1)
(('presentations', 'countries', 'quality'), 1)
(('youtube', 'cost', 'thailand'), 1)
(('seeking', 'entire', 'tr

(('sale', 'frisco', 'root'), 1)
(('recording', 'dish', 'alzheimer'), 1)
(('year', 'bone', 'grafting'), 1)
(('motor', 'accepting', 'applications'), 1)
(('smile', 'lost', 'tooth'), 1)
(('instrument', 'totalimplant', 'sales'), 1)
(('drill', 'instruments', 'coxo'), 1)
(('cancer', 'abutment', 'analog'), 1)
(('healing', 'lost', 'teeth'), 1)
(('ever', 'normally', 'save'), 1)
(('crowns', 'toyota', 'prius'), 1)
(('month', 'book', 'free'), 1)
(('jobs', 'jobsearch', 'coxo'), 1)
(('best', 'perfect', 'smile'), 1)
(('analog', 'slim', 'platform'), 1)
(('insurance', 'plastic', 'lense'), 1)
(('tooth', 'four', 'ashes'), 1)
(('offer', 'manufacturer', 'supplier'), 1)
(('pakistan', 'abutment', 'analog'), 1)
(('screw', 'plastic', 'castable'), 1)
(('nobel', 'cost', 'adidas'), 1)
(('ebaydailybids', 'case', 'wondering'), 1)
(('healthy', 'health', 'dentist'), 1)
(('looking', 'tooth', 'idea'), 1)
(('model', 'nails', 'study'), 1)
(('success', 'predi', 'maintaining'), 1)
(('insurance', 'medical', 'sales'), 1)
(('i

(('abutments', 'high', 'performance'), 1)
(('handpiece', 'surgical', 'drill'), 1)
(('talks', 'favorable', 'might'), 1)
(('ship', 'aaymed', 'promotion'), 1)
(('quality', 'impla', 'absolutely'), 1)
(('organization', 'call', 'month'), 1)
(('analog', 'surgery', 'handpiece'), 1)
(('insurance', 'free', 'online'), 1)
(('offer', 'save', 'full'), 1)
(('farm', 'insurance', 'vegas'), 1)
(('insurance', 'cover', 'problems'), 1)
(('tooth', 'idea', 'available'), 1)
(('insurance', 'business', 'online'), 1)
(('calzedonia', 'bikini', 'online'), 1)
(('base', 'check', 'xive'), 1)
(('brave', 'patient', 'technology'), 1)
(('prosthetic', 'abutment', 'analog'), 1)
(('instrument', 'idea', 'cost'), 1)
(('options', 'average', 'cost'), 1)
(('check', 'discounts', 'dentalimplants'), 1)
(('sales', 'manager', 'folkestone'), 1)
(('azdent', 'handpiece', 'surgery'), 1)
(('interest', 'surgeon', 'looong'), 1)
(('sale', 'costa', 'rica'), 1)
(('implantology', 'completing', 'smile'), 1)
(('thirty', 'charged', 'stealing'), 1)

(('retail', 'addition', 'medicines'), 1)
(('losing', 'tooth', 'talking'), 1)
(('bento', 'tools', 'online'), 1)
(('jobs', 'abutment', 'analog'), 1)
(('share', 'evaluation', 'marginal'), 1)
(('average', 'cost', 'motor'), 1)
(('whatts', 'think', 'someone'), 1)
(('health', 'insurance', 'delaware'), 1)
(('handpiece', 'speed', 'conra'), 1)
(('discount', 'late', 'year'), 1)
(('listings', 'screw', 'placement'), 1)
(('anatomic', 'angular', 'abutment'), 1)
(('adidas', 'shoes', 'greece'), 1)
(('ranking', 'somewhere', 'around'), 1)
(('bachelor', 'resort', 'bend'), 1)
(('learned', 'sheet', 'nearly'), 1)
(('exceed', 'available', 'expectations'), 1)
(('price', 'interested', 'sales'), 1)
(('plans', 'sitting', 'insurance'), 1)
(('consider', 'important', 'understand'), 1)
(('imagine', 'visualize', 'create'), 1)
(('abutments', 'free', 'ship'), 1)
(('boxing', 'classes', 'insurance'), 1)
(('combo', 'insurance', 'covers'), 1)
(('abutment', 'analog', 'kits'), 1)
(('tooth', 'idea', 'arlington'), 1)
(('germany

(('sales', 'base', 'crown'), 1)
(('done', 'pushed', 'behind'), 1)
(('nails', 'study', 'teach'), 1)
(('omegadentalspecialists', 'dentist', 'survived'), 1)
(('speed', 'sandent', 'reduction'), 1)
(('abutment', 'analog', 'repair'), 1)
(('growth', 'dentistry', 'ebay'), 1)
(('check', 'link', 'info'), 1)
(('conta', 'angle', 'system'), 1)
(('covers', 'contemporary', 'center'), 1)
(('handpiece', 'free', 'seminar'), 1)
(('astratech', 'osseospeed', 'healing'), 1)
(('ebaydailybids', 'ensure', 'abutments'), 1)
(('made', 'parts', 'crown'), 1)
(('questions', 'throughout', 'june'), 1)
(('visit', 'call', 'month'), 1)
(('bait', 'switch', 'demonstration'), 1)
(('handpiece', 'style', 'types'), 1)
(('involves', 'using', 'four'), 1)
(('jobsearch', 'motor', 'reduction'), 1)
(('philippines', 'online', 'shopping'), 1)
(('sizes', 'azdent', 'motor'), 1)
(('oral', 'surgery', 'idea'), 1)
(('ideas', 'long', 'term'), 1)
(('take', 'year', 'bone'), 1)
(('insurance', 'denials', 'appeals'), 1)
(('four', 'seasons', 'loan

(('high', 'schools', 'miami'), 1)
(('optic', 'check', 'frios'), 1)
(('handpiece', 'cica', 'surgical'), 1)
(('clark', 'dispenser', 'used'), 1)
(('sizes', 'video', 'lead'), 1)
(('angle', 'handpiece', 'tooth'), 1)
(('lectures', 'complete', 'surgery'), 1)
(('savannah', 'surgery', 'stemming'), 1)
(('today', 'need', 'bottom'), 1)
(('blog', 'dentist', 'tijuana'), 1)
(('free', 'quote', 'robot'), 1)
(('sites', 'across', 'london'), 1)
(('sale', 'fortis', 'straight'), 1)
(('determination', 'services', 'raleigh'), 1)
(('listings', 'takes', 'months'), 1)
(('introducing', 'permanent', 'replacing'), 1)
(('angle', 'immune', 'disease'), 1)
(('sale', 'styl', 'reduction'), 1)
(('dentures', 'thailand', 'cost'), 1)
(('forc', 'piece', 'angular'), 1)
(('oral', 'surgery', 'wondering'), 1)
(('macrogap', 'dentures', 'save'), 1)
(('bling', 'party', 'surgery'), 1)
(('abutment', 'internal', 'sandent'), 1)
(('health', 'insurance', 'deductions'), 1)
(('house', 'rental', 'medicaid'), 1)
(('free', 'adblock', 'chrome')

(('jersey', 'meet', 'behind'), 1)
(('scott', 'keith', 'presents'), 1)
(('bahamas', 'abaco', 'island'), 1)
(('abutment', 'tooth', 'influence'), 1)
(('crowns', 'crown', 'attached'), 1)
(('retrieval', 'method', 'nobel'), 1)
(('visit', 'nobel', 'biocare'), 1)
(('replace', 'tooth', 'lost'), 1)
(('brush', 'straight', 'narrow'), 1)
(('invention', 'ideas', 'reduction'), 1)
(('yusendent', 'addition', 'medicines'), 1)
(('around', 'children', 'savings'), 1)
(('jobsearch', 'abutment', 'analog'), 1)
(('insurance', 'waiting', 'period'), 1)
(('kavo', 'ebaydailybids', 'ensure'), 1)
(('sending', 'brand', 'contra'), 1)
(('exams', 'idea', 'cost'), 1)
(('amazing', 'complete', 'smile'), 1)
(('supplier', 'whatsapp', 'pilot'), 1)
(('load', 'money', 'costa'), 1)
(('insurance', 'best', 'sites'), 1)
(('angle', 'reduction', 'take'), 1)
(('surgery', 'wondering', 'friday'), 1)
(('coverage', 'skin', 'cancer'), 1)
(('around', 'months', 'importan'), 1)
(('tools', 'manufacturers', 'exporters'), 1)
(('manufacturers', '